In [4]:
import os, sys; sys.path.append(os.path.dirname(os.getcwd()))

In [37]:
import torch
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt

import util.RAVDESS_dataset_util as Rd
import multimodal_vae
import train_mvae_plain

In [38]:
class Config_model_args:
    img_size= 64
    z_dim= 256
    hidden_dim= 512
    loss_weights = {'face': 1.0,'emocat': 1.0}
    expert_type= "moe"
    dataset_path= '/home/studenti/ballerini/datasets/RAVDESS_frames_ds'


class Config_train_args:
    learning_rate= 0.0001
    optim_betas= [ 0.95, 0.98 ]
    num_epochs= 100
    batch_size= 32,
    checkpoint_every= 20
    checkpoint_path= "./"
    save_model= True
    model_save_path= "trained_models/ravdess_mmvae_pretrained_plain.pt",
    stats_save_path= "trained_models/ravdess_mmvae_pretrained_plain_stats.pt",
    seed= 100
    use_cuda= True
    annealing_type= "static"
    cyclical_annealing= {
      'min_beta': 0.0001,
      'max_beta': 0.8,
      'num_cycles': 8,
      'annealing_percentage': 0.9
    }
    linear_annealing= {
      'min_beta': 0.001,
      'max_beta': 1.0,
    }
    static_annealing_beta= 0.1


cfg_model = Config_model_args()
cfg_train = Config_train_args()

In [39]:
face_dataset = Rd.FaceEmotionDataset(
    root_dir=cfg_model.dataset_path,
    transform=transforms.Compose
    ([
        Rd.Rescale(cfg_model.img_size), 
        Rd.CenterCrop(cfg_model.img_size), 
        Rd.ToTensor()
    ]))

trainingset_len = len(face_dataset) // 100 * 90
testset_len = len(face_dataset) - trainingset_len
training_dataset, testing_dataset = torch.utils.data.random_split(face_dataset, 
                                                    [trainingset_len, testset_len], 
                                                    #generator=torch.Generator().manual_seed(42)
                                                   )

In [41]:
# img = np.array(next(iter(face_dataset))['image']).transpose((1, 2, 0))
# print(img.shape)
# plt.imshow(img)

In [ ]:
training_dataset, testing_dataset = util.load_preprocessed_dataset(cfg=cfg)

input_dims: dict = util.get_modality_input_dimensions_from_data(
    dataset=training_dataset,
    modality="plain"
)
    
model: torch.nn.Module = build_model(
    input_dims=input_dims,
    latent_space_dim=cfg.model.plain.latent_space_dim,
    va_mlp_hidden_dim=cfg.model.plain.va_mlp_hidden_dim,
    loss_weights=cfg.model.plain.loss_weights,
    expert_type=cfg.model.plain.expert_type,
    use_cuda=cfg.train.plain.use_cuda
)
    
train(
    mmvae_model=model,
    dataset=training_dataset,
    learning_rate=cfg.train.plain.learning_rate,
    optim_betas=cfg.train.plain.optim_betas,
    num_epochs=cfg.train.plain.num_epochs,
    batch_size=cfg.train.plain.batch_size,
    checkpoint_every=cfg.train.plain.checkpoint_every,
    checkpoint_path=cfg.train.plain.checkpoint_path,
    save_model=cfg.train.plain.save_model,
    seed=cfg.train.plain.seed,
    use_cuda=cfg.train.plain.use_cuda,
    cfg=cfg
)
